In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (15.0, 4.0) # set default size of plots

In [2]:
orders = pd.read_csv('./instacart_2017_05_01/orders.csv')
orders = orders[orders.eval_set=='prior'].drop('eval_set',axis=1)

aisles = pd.read_csv('./instacart_2017_05_01/aisles.csv')
prods = pd.read_csv('./instacart_2017_05_01/products.csv')

In [3]:
aisles['aisle'] = aisles.aisle.str.split().apply(lambda x: '_'.join(x))

In [4]:
ord_aisle = pd.merge(
    pd.read_csv('./instacart_2017_05_01/order_products__prior.csv',nrows=1000000),
    pd.merge(
        prods[['product_id','aisle_id']], aisles,
        on='aisle_id',how='left'
    ),
    on='product_id',
    how='left'
)[['order_id','add_to_cart_order','reordered','aisle']]

In [5]:
ord_aisle = pd.get_dummies(ord_aisle,columns=['aisle'],prefix='aisle').drop(['reordered'],axis=1)

In [6]:
ord_aisle.drop(['add_to_cart_order'],axis=1,inplace=True)
ord_aisle = ord_aisle.groupby('order_id').sum()
ord_aisle.reset_index(inplace=True)

In [7]:
res = pd.merge(orders[['order_id','user_id']],ord_aisle,on='order_id',how='right')

In [13]:
res

,order_id,user_id,aisle_air_fresheners_candles,aisle_asian_foods,aisle_baby_accessories,aisle_baby_bath_body_care,aisle_baby_food_formula,aisle_bakery_desserts,aisle_baking_ingredients,aisle_baking_supplies_decor,...,aisle_spreads,aisle_tea,aisle_tofu_meat_alternatives,aisle_tortillas_flat_bread,aisle_trail_mix_snack_mix,aisle_trash_bags_liners,aisle_vitamins_supplements,aisle_water_seltzer_sparkling_water,aisle_white_wines,aisle_yogurt
0,94891,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,23391,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,68288,10,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,83395,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,19256,13,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
5,86918,19,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,62373,21,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,2
7,77791,21,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
8,8382,23,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,4
9,65724,27,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [22]:
X = np.zeros((100,134,4),dtype='uint8')

In [17]:
users = res.user_id.unique() 

In [21]:
res[res.user_id == users[2]]

,order_id,user_id,aisle_air_fresheners_candles,aisle_asian_foods,aisle_baby_accessories,aisle_baby_bath_body_care,aisle_baby_food_formula,aisle_bakery_desserts,aisle_baking_ingredients,aisle_baking_supplies_decor,...,aisle_spreads,aisle_tea,aisle_tofu_meat_alternatives,aisle_tortillas_flat_bread,aisle_trail_mix_snack_mix,aisle_trash_bags_liners,aisle_vitamins_supplements,aisle_water_seltzer_sparkling_water,aisle_white_wines,aisle_yogurt
2,68288,10,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,83395,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [59]:
from keras.layers import Dense,Conv1D,LSTM,concatenate,Input,Reshape

In [65]:
x = Input(shape=(4,134))
x1 = Conv1D(20,kernel_size=(10),padding='same',activation='relu')(x)
x2 = Conv1D(20,kernel_size=(50),padding='same',activation='relu')(x)
x3 = Conv1D(30,kernel_size=(100),padding='same',activation='relu')(x)
x_c = concatenate([x1,x2,x3])
y = Input(shape=(4,1))
out = concatenate([x_c,y])

In [66]:
from keras.models import Model

In [67]:
model = Model(inputs=[x,y],outputs=out)

In [68]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_14 (InputLayer)            (None, 4, 134)        0                                            
____________________________________________________________________________________________________
conv1d_5 (Conv1D)                (None, 4, 20)         26820       input_14[0][0]                   
____________________________________________________________________________________________________
conv1d_6 (Conv1D)                (None, 4, 20)         134020      input_14[0][0]                   
____________________________________________________________________________________________________
conv1d_7 (Conv1D)                (None, 4, 30)         402030      input_14[0][0]                   
___________________________________________________________________________________________